In [1]:
!pip install scrapy

import json
import logging
from urllib.parse import parse_qs, urlparse
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess


class BookingSpider(scrapy.Spider):
    name = 'booking_data'
    top_cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

    list_url = ['https://www.booking.com/searchresults.fr.html?aid=304142&ss={}&offset='.format(n) for n in top_cities]
    start_urls = []
    for n in list_url:
        for i in range(0,25):
            start_urls.append(str(n) + str(i))
    
    def start_requests(self):
        top_cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]
        z = 0
        for url in self.start_urls:
            yield scrapy.Request(url, self.parse , meta ={"city" :top_cities[z] })
            z+=1


    def parse(self, response):
        rows = response.xpath("//div[@data-testid = 'property-card']")
      
        for row in rows:
            hotel_name = row.xpath('.//div[@data-testid = "title"]/text()').get()
            url_hotel = row.xpath('.//h3[@class = "a4225678b2"]/a/@href').get()
            hotel_score = row.xpath('.//div[@class = "b5cd09854e d10a6220b4"]/text()').get()
            city = response.request.meta["city"]

            yield response.follow(url= url_hotel, callback=self.parse_hotel, meta={"hotel_name": hotel_name, "hotel_score": hotel_score, "url_hotel": url_hotel, "city":city})
        

    def parse_hotel(self, response):
        url_hotel = response.request.meta['url_hotel']
        hotel_name = response.request.meta['hotel_name']
        hotel_score = response.request.meta['hotel_score']
        city = response.request.meta['city']
        gps = response.xpath('/html/body/script[26]/text()').get()
        lat = float((gps.split(' ')[2].split(';'))[0])
        lon = float((gps.split(' ')[4].split(';'))[0])
        description = ''
        for row in response.xpath('//div[@id = "property_description_content"]/p'):

            description += str((row.xpath('.//text()').get()))
        yield {
                "hotel_name": hotel_name,
                "description": description,
                "city": city,
                "hotel_score": hotel_score,
                "url_hotel": url_hotel,
                "lat": lat,
                "lon": lon
                
            }


process = CrawlerProcess( {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.110 Safari/537.36',
    "FEEDS": {
       "hotel" : {"format": "csv"},
    }
})


# Name of the file where the results will be saved
filename = "data_hotelsV2.json"

# If file already exists, delete it before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir('./src/'):
        os.remove('./src/' + filename)
        

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        './src/' + filename: {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.3 MB/s eta 0:00:00


INFO:scrapy.utils.log:Scrapy 2.9.0 started (bot: scrapybot)
2023-06-12 19:33:48 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform Linux-5.15.107+-x86_64-with-glibc2.31
2023-06-12 19:33:48 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform Linux-5.15.107+-x86_64-with-glibc2.31
INFO:scrapy.utils.log:Scrapy 2.9.0 started (bot: scrapybot)
2023-06-12 19:33:48 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.